In [ ]:
!nvidia-smi

# Setup

In [ ]:
!yes | pip uninstall tensorflow

!pip install tensorflow-gpu==1.13.1 humanfriendly jsonpickle

import tensorflow as tf
print(tf.__version__)

!wget -O /content/megadetector_v4_1_0.pb https://lilablobssc.blob.core.windows.net/models/camera_traps/megadetector/md_v4.1.0/md_v4.1.0.pb

!git clone https://github.com/microsoft/CameraTraps
!git clone https://github.com/microsoft/ai4eutils

!cp /content/CameraTraps/detection/run_tf_detector_batch.py .
!cp /content/CameraTraps/visualization/visualize_detector_output.py .

import json
import os
import shutil
from pathlib import Path

from tqdm import tqdm

os.environ['PYTHONPATH'] += ":/content/ai4eutils"
os.environ['PYTHONPATH'] += ":/content/CameraTraps"
!echo "PYTHONPATH: $PYTHONPATH"

# Get Raw Data

In [ ]:
#@title Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

google_drive_folder_name = 'sample' #@param {type: "string"}
images_dir = '/content/drive/My Drive/' + google_drive_folder_name

!ls "$images_dir"

Path(f'{images_dir}/output').mkdir(exist_ok=True)
Path(f'{images_dir}/output/no_detections').mkdir(exist_ok=True)
Path(f'{images_dir}/output/with_detections_and_bb').mkdir(exist_ok=True)
Path(f'{images_dir}/output/with_detections').mkdir(exist_ok=True)

# Run The Model

In [ ]:
# choose a location for the output JSON file
output_file_path = f'{images_dir}/output' + '/data.json'
!python run_tf_detector_batch.py megadetector_v4_1_0.pb "$images_dir" "$output_file_path" --recursive

# Get The Results

In [ ]:
visualization_dir = '/content/viz'  # pick a location for annotated images

!python visualize_detector_output.py "$output_file_path" "$visualization_dir" --confidence 0.01 --images_dir "$images_dir"

In [ ]:
def categorize(string):
  return string.replace('1', 'animal').replace('2', 'person').replace('3', 'vehicle')


with open(output_file_path) as j:
  data = json.load(j)

%cd CameraTraps
from data_management.annotations.annotation_constants import (
    detector_bbox_category_id_to_name)
from visualization import visualization_utils as vis_utils
%cd ..

In [ ]:
Path(f'{images_dir}/output/no_detections').mkdir(exist_ok=True)

display_images_here = False #@param {type: "boolean"}
if display_images_here:
  if len(data['images']) > 20:
    print('There are too many images to display! View the images on Google Drive.')
    display_images_here = False

copy_images_to_drive = False #@param {type: "boolean"}

for image in tqdm(data['images']):
    if not image['detections']:
        im = vis_utils.resize_image(
                vis_utils.open_image(image['file']), 700)
        if display_images_here:
            display(im)
        if copy_images_to_drive:
            out_path = f'{images_dir}/output/no_detections/{Path(image["file"]).name}'
            if not Path(out_path).exists():
                shutil.copy2(image['file'], out_path)

In [ ]:
Path(f'{images_dir}/output/with_detections_and_bb').mkdir(exist_ok=True)
Path(f'{images_dir}/output/with_detections').mkdir(exist_ok=True)

min_detection_conf_to_save = "0.5" #@param {type: "string"}

display_images_here = False #@param {type: "boolean"}
if display_images_here:
    if len(data['images']) > 20:
        print('There are too many images to display! View the images on Google Drive.')
        display_images_here = False

copy_images_to_drive = False #@param {type: "boolean"}

for image in data['images']:
  if image['detections']:
    if image['max_detection_conf'] >= float(min_detection_conf_to_save):
        print('-' * 79)
        print(image['file'])
        res = [(categorize(x['category']), x['conf']) for x in image['detections']]
        for n, x in enumerate(res):
            print(f'{n + 1}. {x[0]} (conf: {x[1]})')

        img_file = visualization_dir + '/anno_' + images_dir.replace('/', '~') + '~' + Path(image['file']).name
        im = vis_utils.resize_image(vis_utils.open_image(img_file), 700)
        
        if display_images_here:
            display(im)

        if copy_images_to_drive:
            out_path_with_bb = f'{images_dir}/output/with_detections_and_bb/{Path(img_file).name}'
            if not Path(out_path_with_bb).exists():
                shutil.copy2(img_file, out_path_with_bb)
            out_path = f'{images_dir}/output/with_detections/{Path(image["file"]).name}'
            if not Path(out_path).exists():
                shutil.copy2(image["file"], out_path)
